<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-041221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import plotly.express as px

# StandardScaler

In [2]:
x=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
# x=np.array(x)
scaler = preprocessing.StandardScaler().fit(x)
print(scaler.mean_, scaler.scale_)
x_scaled = scaler.transform(x)
print (x_scaled)
x_back=scaler.inverse_transform(x_scaled)
print (x_back)
scaler.inverse_transform([[0,0,0]])
# help(scaler)

[ 0.          0.33333333 -0.16666667] [1.15470054 0.74535599 0.68718427]
[[ 1.73205081  0.89442719  0.24253563]
 [-0.8660254  -0.4472136   1.69774938]
 [ 0.         -0.4472136   0.24253563]
 [ 0.8660254   0.89442719  0.24253563]
 [-0.8660254  -1.78885438 -1.21267813]
 [-0.8660254   0.89442719 -1.21267813]]
[[ 2.  1.  0.]
 [-1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1. -1. -1.]
 [-1.  1. -1.]]


array([[ 0.        ,  0.33333333, -0.16666667]])

# Single Neuron

In [3]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)
        if self.activ == 'mrelu':
          return self.mrelu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init, dtype='double')

    def set_activ(self, activ, param=0):
        """Overrides the 'linear' activation function."""
        self.activ = activ
        self.param = param

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

    def mrelu(self, x):
        # return the output of the modified relu function applied to x
        if x >= 0:
          return x
        return self.param*x

In [4]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
neu.set_activ('mrelu', param=0.01)
neu.run([1,0])

-0.05

## AND gate

In [5]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [6]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [7]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [8]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ, param=0):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ, param) # set_activ for each neuron
        self.param=param
    
    def set_output_activ(self, activ, param=0):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ, param) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0
        if self.network[i][j].activ == 'mrelu':
          if value > 0:
            return 1
          else:
            return self.param

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [9]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10. -10.  15.] sigmoid
Layer 1 Neuron 1 : [ 15.  15. -10.] sigmoid
Layer 2 Neuron 0 : [ 10.  10. -15.] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [10]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.1736465000551295
0.9900365227548784
0.5293792037784211
0.04905131650170966
0.020156047087472113
0.012237049175974688
0.008670382284526174
0.0066698511372297485
0.005398880412965443
0.004523735597147044

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.38609956  4.38074454 -6.81889215] sigmoid
Layer 1 Neuron 1 : [-6.03174576 -6.02025344  2.30435949] sigmoid
Layer 2 Neuron 0 : [-8.8423013  -8.63175111  4.30896357] sigmoid

XOR Gate:
0 0 = 0.0279496955
0 1 = 0.9675098648
1 0 = 0.9674715812
1 1 = 0.0313549046


# Regres

In [11]:
class Regres:
    def __init__(self, layers, bias=1.0, eta=0.1, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ, param=0):
        self.regres_network.set_activ(activ, param=0)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y, epochs):
        self.epochs=epochs
        self.weight_history=[]
        self.weight_history_table=[]
        # self.cols=[]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            weight_epoch_table=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]])
            MSE = MSE / len(X)
            self.MSE_history.append(MSE)
            for m in range(1,len(self.layers)):
                for n in range(self.layers[m]):
                    # col="{}_{}".format(m,n)
                    # self.cols.append(col)
                    neuron_w=self.regres_network.network[m][n].weights
                    neuron_w_list=[x for x in neuron_w]
                    weight_epoch.append(neuron_w_list)
                    weight_epoch_table+=neuron_w_list
            self.weight_history.append(weight_epoch)
            self.weight_history_table.append(weight_epoch_table)
        self.weight_history_table=pd.DataFrame(data=self.weight_history_table, columns=self.get_cols())
        print ("""Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation""")

    def get_cols(self): 
        cols=[]
        for i in range(1,len(self.layers)):
            for h in range(self.layers[i]):
                for k in range(self.layers[i-1]+1): 
                    col="{}_{}_{}".format(i,h,k) 
                    cols.append(col)
        return cols 
    
    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

In [12]:
weight_history=[]
a=[[1,2],[1,3],[4,1]]
b=[]
for m in range(3):
  for n in range(2):
    b.append(a[m][n])
print ("b",b)
weight_history.append(b)
print (weight_history)

b [1, 2, 1, 3, 4, 1]
[[1, 2, 1, 3, 4, 1]]


# Regres_scale

In [13]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
class Regres_scale:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        # self.epochs = epochs      
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, x, y, epochs):
        self.scaler = preprocessing.StandardScaler().fit(x)
        x_scaled=self.scaler.transform(x)
        self.epochs=epochs
        for i in range(epochs):
            MSE = 0.0
            for j in range (len(x)):
                MSE +=  self.regres_network.bp_regres(x_scaled[j],[y[j]])
            MSE = MSE / len(x)
            # dobavit' istoriyu kak v Regres

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

    def get_standard_params(self, x):
        x2=np.array(x)
        mean=x2.mean(axis=0)
        stdev=x2.std(axis=0)
        return mean, stdev

# Example 1 -- weights

In [14]:
print ("1 1 =",mln1.run([1,1]))
# print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"
w1=mln1.network[1][0].weights
w2=[x for x in w1]
w2

1 1 = [0.00715281]


[-10.0, -10.0, 15.0]

In [15]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.77771773 -0.28611995 -0.20367542] relu
Layer 1 Neuron 1 : [-0.89104011  0.90529975  0.88511192] relu
Layer 2 Neuron 0 : [-0.88973595  0.08911785 -0.81835596] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.77771773 -0.28611995 -0.20367542] relu
Layer 1 Neuron 1 : [-0.89104011  0.90529975  0.88511192] relu
Layer 2 Neuron 0 : [-0.88973595  0.08911785 -0.81835596] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.77771773 -0.28611995 -0.20367542] linear
Layer 1 Neuron 1 : [-0.89104011  0.90529975  0.88511192] relu
Layer 2 Neuron 0 : [-0.88973595  0.08911785 -0.81835596] sigmoid



# Example 3 -- linear activs, bp_regres. מתכנס למשקלים טוב מאוד 

In [16]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0,0,0,0]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],5.764286995)
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],5.84646758)
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],0.808633075)
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],3.850298589)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

54.560573414122615
9.652321993077698e-13

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253294e-05] linear



# Example 4 -- sigmoid, bp_regres. מקרב רגרסיה עם סיגמויד. עובד טוב

In [17]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.1)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] sigmoid
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] sigmoid
Layer 2 Neuron 0 : [-1.  2.  3.] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 14.832802157292896
MSE= 0.008246981713727427
MSE= 0.005589985703685294
MSE= 0.003958147954843152
MSE= 0.0033339469478493896

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.84329924 -2.06273176  0.76204796  0.7546715 ] sigmoid
Layer 1 Neuron 1 : [ 1.33755264 -2.19888628  1.66989995  0.25943488] sigmoid
Layer 2 Neuron 0 : [2.03601701 0.3591475  2.1670272 ] linear



'predicted values:'

[3.6182448006828714,
 3.746627106646207,
 3.7546551231557097,
 3.0797442726505864,
 3.867580917572126,
 2.274405263859319]

# Example 5 -- relu-sigmoid, bp_regres. מתכנס למשקלים לא משהו

In [18]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%20 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] relu
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] relu
Layer 2 Neuron 0 : [1. 0. 1.] sigmoid
Layer 3 Neuron 0 : [-1.  2.] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 0.35444956933566835
MSE 20 = 0.04879122981326257
MSE 40 = 0.048462748319034056
MSE 60 = 0.04660751008106884
MSE 80 = 0.042866203917052266
MSE 100 = 0.035454201918272685
MSE 120 = 0.02083971548026657
MSE 140 = 0.0054918209205328
MSE 160 = 0.0010642353269303424
MSE 180 = 0.0005504873066299159
MSE 200 = 0.00045535862569053546
MSE 220 = 0.00041640696704562974
MSE 240 = 0.00039757708445461526
MSE 260 = 0.00038825956133610435
MSE 280 = 0.00038370832552052933
MSE 300 = 0.00038164841965195365
MSE 320 = 0.0003809404213891736
MSE 340 = 0.00038098737996475145
MSE 360 = 0.0003814623001760395
MSE 380 = 0.0003821784753779646

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.19403984  0.31138897 -0.87010575  1.03754352] relu
Layer 1 Neuron 1 : [-0.42293304 -0.27062983 -0.60284145  0.9228103 ] relu
Layer 2 Neuron 0 : [-0.92987578 -0.0726913  -0.07354992] sigmoid
Layer 3 Neuron 0 : [0.71945489 0.92512971] linear



'predicted values:'

[1.0373211972100707,
 1.2619461502451268,
 1.1040770766417007,
 1.0537166968891514,
 1.0522687826697275,
 1.00513467004659]

# Example 7 -- Regres class. עובד באופן עקרוני
# relu בעייתי

In [19]:
reg1=Regres(layers=[3,2,1,1], eta=0.3)
reg1.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y, epochs=5)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
display ('MSE',reg1.MSE_history)

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 9.99713140e-01  1.00042202e+00 -2.75446741e-04  7.33518678e-04] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.03277788  0.          0.00458208] sigmoid
Layer 3 Neuron 0 : [0.40593827 0.81473119] linear

[1.0081910251075497, 1.0181653384338307, 1.0181628984344357, 1.0115114716378777, 1.0181653384338307, 1.0181596241520412]



'MSE'

[0.8936248829442258,
 0.04857883327236249,
 0.0481469881158581,
 0.04777680393002976,
 0.04744442311981101]

# Example -- Regres_scale. צריך חשוב ולהשלים

In [20]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
reg2=Regres_scale(layers=[3,2,1,1], eta=0.1)
reg2.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg2.set_hidden_activ('relu')
reg2.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg2.fit(X,y,epochs=200)
reg2.printWeights()
pred=reg2.predict(X)
pred


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.9517462   1.06201428 -0.06407585  0.06933407] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.20894407  0.          0.00917428] sigmoid
Layer 3 Neuron 0 : [0.46600036 0.84566849] linear



[1.0081438427807037,
 1.0797374688505368,
 1.078049747612554,
 1.029768336904302,
 1.0797374688505368,
 1.073806571646052]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [21]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(100):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4
    if(i%10 == 0):
        print ('MSE', i, '=',MSE)

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear

MSE 0 = 3.097759865123268
MSE 10 = 0.2948017967667316
MSE 20 = 0.11235016703947856
MSE 30 = 0.039031749833149484
MSE 40 = 0.005346702764168406
MSE 50 = 0.00047155150910413527
MSE 60 = 9.822149422722454e-05
MSE 70 = 6.0504726410544755e-05
MSE 80 = 4.67746394940529e-05
MSE 90 = 3.966114337742912e-05

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.92948745 -0.3432972  -0.4785581   0.21811547 -0.0456272  -1.65606983] sigmoid
Layer 1 Neuron 1 : [-1.01311017 -0.23274925 -1.57166574  1.18384536  0.1929425  -1.78487787] sigmoid
Layer 1 Neuron 2 : [-0.09277171 -0.3986799   1.19699659 -0.59778185 -0.68424799 -2.10503325] sigmoid
Layer 2 Neuron 0 : [ 0.25752661  0.97942477 -1.06640139  0.35525422] linear



# Example 8 -- Regres_linear. עובד טוב

In [22]:
reg2=Regres(layers=[1,2,1], eta=0.01)
X=[[0],[1],[2],[3],[4],[5]]
y=[0,1,2,3,4,5]
reg2.printWeights()

reg2.set_hidden_activ('linear')
reg2.fit(X,y,epochs=5)
reg2.printWeights()
pred=reg2.predict(X)
print ('predictions',pred)
display ('weight_history_table',reg2.weight_history_table)
display ('MSE',reg2.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.50354794 -0.3249484 ] linear
Layer 1 Neuron 1 : [-0.907794    0.76462787] linear
Layer 2 Neuron 0 : [-0.10758018 -0.98543248 -0.59433049] linear

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.52253631 -0.37297147] linear
Layer 1 Neuron 1 : [-1.04652363  0.51755252] linear
Layer 2 Neuron 0 : [-0.26496005 -0.95668998 -0.34609788] linear

predictions [-0.7424126593285164, 0.39723726416982963, 1.5368871876681756, 2.6765371111665224, 3.8161870346648676, 4.955836958163214]


'weight_history_table'

,1_0_0,1_0_1,1_1_0,1_1_1,2_0_0,2_0_1,2_0_2
0,-0.525365,-0.338221,-1.050163,0.666545,-0.214307,-1.050556,-0.494249
1,-0.524077,-0.347398,-1.047357,0.622350,-0.227398,-1.019399,-0.451818
2,-0.523611,-0.356384,-1.047644,0.582767,-0.241304,-0.995426,-0.412688
3,-0.523091,-0.364912,-1.047287,0.548110,-0.253746,-0.974710,-0.377636
4,-0.522536,-0.372971,-1.046524,0.517553,-0.264960,-0.956690,-0.346098


'MSE'

[4.0233317645176125,
 1.6398958876779222,
 1.3390428184008767,
 1.0849280381089288,
 0.8854400913762439]

# Example 9 -- mrelu check. working

MultiLayerNeuron

In [23]:
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.01)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')
mln4.set_weights([[[-1.1,1.74],[3.35,-6.44]],[[-1.41,2.05,2.2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[1,2,1], eta=0.001) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.01)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')

for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([0],y4[0])
    MSE += mln4.bp_regres([1],y4[1])
    MSE += mln4.bp_regres([2],y4[2])
    MSE += mln4.bp_regres([3],y4[3])
    MSE += mln4.bp_regres([4],y4[4])
    MSE += mln4.bp_regres([5],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.1   1.74] mrelu
Layer 1 Neuron 1 : [ 3.35 -6.44] mrelu
Layer 2 Neuron 0 : [-1.41  2.05  2.2 ] linear



'original values:'

[-0.38541999999999943,
 1.2342550000000005,
 2.7394859999999994,
 9.622496,
 16.505506,
 23.388515999999996]


MSE 0 = 300.10594795761125
MSE 200 = 9.62255870510367
MSE 400 = 9.091754914029192
MSE 600 = 2.971609649962556
MSE 800 = 1.2503686645753598
MSE 1000 = 0.9051929320201468
MSE 1200 = 0.8261237628543127
MSE 1400 = 0.8080908651940275
MSE 1600 = 0.8040035023367545
MSE 1800 = 0.803085881517667
MSE 2000 = 0.8028828749525937
MSE 2200 = 0.8028383828285941
MSE 2400 = 0.8028278132126491
MSE 2600 = 0.8028238898756408
MSE 2800 = 0.8028209500849375
MSE 3000 = 0.8028179719081526
MSE 3200 = 0.8028148516826611
MSE 3400 = 0.802811627704659
MSE 3600 = 0.8028083382918663
MSE 3800 = 0.8028050062786186

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.93684564 -3.24633695] mrelu
Layer 1 Neuron 1 : [-0.1827512 -0.0052491] mrelu
Layer 2 Neuron 0 : [ 3.5540612  -0.27201426  0.50588813] linear



'predicted values:'

[0.3905256039928392,
 0.45985939262216824,
 2.7365522841128604,
 9.620717324398973,
 16.504882364685088,
 23.3890474049712]

# Example 10 - Regres_mrelu. working

In [100]:
reg2=Regres(layers=[1,2,1], eta=0.0001)
X=[[-3],[-2],[-1],[0],[1],[2],[3],[4],[5]]
y=[9,4,1,0,1,4,9,16,25]
# y=[0,1,2,3,4,5]
reg2.set_hidden_activ('mrelu', param=0.01)
# reg2.set_weights([[[0,0],[0,0]],[[0,0,0]]])
reg2.fit(X,y,epochs=5000)
reg2.printWeights()
pred=reg2.predict(X)
print ('predictions',pred)
# display ('weight_history_table',reg2.weight_history_table.iloc[-1])
# display ('MSE',reg2.MSE_history)

fig=px.line(y=reg2.MSE_history[500:])
fig.show()

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - pandas table of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.68288889 -1.0683856 ] mrelu
Layer 1 Neuron 1 : [ 1.96902717 -3.06606325] mrelu
Layer 2 Neuron 0 : [ 2.12083687  3.55348246 -0.06372082] linear

predictions [8.377806072566921, 4.808673252386137, 1.239540432205354, -0.06372082027359174, -0.06372082027359174, 3.03488422935104, 10.031787743440763, 17.028691257530486, 24.02559477162021]


In [39]:
history=reg2.weight_history_table.T
# history
fig=px.parallel_coordinates(history)
fig.show()